##  Introdução

Neste projeto, foram analisados dados de imóveis da cidade de São Paulo com o objetivo de entender os principais fatores que influenciam o preço dos imóveis, utilizando técnicas de regressão linear e análise estatística.

O projeto contempla análise exploratória, modelagem, diagnóstico e sugestões para aprimoramento do modelo.

## Carregamento e Preparação dos Dados

Aqui foi realizado o carregamento da base de dados, inspeção inicial, e tratamento de valores ausentes ou inconsistentes.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('dados/imoveis_sp.csv')

print("Formato:", df.shape)
print("\nTipos de dados:\n", df.dtypes)
print("\nValores ausentes:\n", df.isnull().sum())
df.head()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
import statsmodels.api as sm


In [ ]:
df = pd.read_csv('dados/imoveis_sp.csv')
df.head()


In [ ]:
df.info()
df.describe()
df.isnull().sum()


In [ ]:
df = df[(df['Price'] > 0) & (df['Size'] > 0)]
df.dropna(subset=['Price', 'Size'], inplace=True)


In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title('Correlação entre Variáveis Numéricas')
plt.tight_layout()
plt.savefig('grafico-correlacao.png') 
plt.show()


In [ ]:
import statsmodels.api as sm

df['Size'] = pd.to_numeric(df['Size'], errors='coerce')
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df.dropna(subset=['Size', 'Price'], inplace=True)

X = df['Size']
y = df['Price']

X = sm.add_constant(X)  
modelo = sm.OLS(y, X).fit()
print(modelo.summary())




In [ ]:
sns.regplot(x='Size', y='Price', data=df, scatter_kws={"alpha":0.3})
plt.title('Regressão Linear: Área vs Preço')
plt.xlabel('Área útil (m²)')
plt.ylabel('Preço (R$)')
plt.savefig('grafico-regressao.png')
plt.show()


In [ ]:

variaveis = ['Size', 'Rooms', 'Toilets', 'Suites', 'Parking']

df[variaveis + ['Price']] = df[variaveis + ['Price']].apply(pd.to_numeric, errors='coerce')

df.dropna(subset=variaveis + ['Price'], inplace=True)

X = df[variaveis]
y = df['Price']

X = sm.add_constant(X)

modelo_multiplo = sm.OLS(y, X).fit()

print(modelo_multiplo.summary())


In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(df['Price'], bins=50, kde=True)
plt.title('Distribuição dos Preços dos Imóveis')
plt.xlabel('Preço (R$)')
plt.ylabel('Frequência')
plt.savefig('imagens/distribuicao-precos.png')
plt.show()

plt.figure(figsize=(10,6))
sns.scatterplot(x='Size', y='Price', hue='Parking', data=df, palette='viridis', alpha=0.6)
plt.title('Preço vs. Área (com Vagas)')
plt.xlabel('Área útil (m²)')
plt.ylabel('Preço (R$)')
plt.legend(title='Vagas')
plt.savefig('imagens/size-vs-price-parking.png')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('dados/imoveis_sp.csv')
df.head()

## Análise Exploratória (EDA)

Nesta seção, é possível visualizar a distribuição dos preços, a correlação entre variáveis e outras informações úteis para entender o comportamento dos dados.

In [ ]:

plt.figure(figsize=(10, 6))
sns.histplot(df['Price'], bins=30, kde=True)
plt.title('Distribuição dos Preços dos Imóveis')
plt.xlabel('Preço (R$)')
plt.ylabel('Frequência')
plt.show()

plt.figure(figsize=(12, 10))
corr = df.select_dtypes(include='number').corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlação entre Variáveis Numéricas')
plt.show()


## Transformações e Engenharia de Variáveis

Aqui foram aplicadas transformações como logaritmo para lidar com distribuições assimétricas, além de outras possíveis transformações úteis para a modelagem.

In [ ]:

df['LogPrice'] = np.log1p(df['Price'])

plt.figure(figsize=(10, 6))
sns.histplot(df['LogPrice'], bins=30, kde=True)
plt.title('Distribuição do Log(Preço)')
plt.xlabel('Log(Preço)')
plt.ylabel('Frequência')
plt.show()


##  Regressão Linear Simples

Exploramos a relação entre a variável de área útil e o preço dos imóveis usando regressão linear simples.

In [ ]:

import statsmodels.api as sm

X = df['Size']
y = df['Price']

X_const = sm.add_constant(X)
modelo_simples = sm.OLS(y, X_const).fit()
print(modelo_simples.summary())

sns.lmplot(x='Size', y='Price', data=df, scatter_kws={"alpha":0.3}, line_kws={"color": "red"})
plt.title('Regressão Linear: Área vs Preço')
plt.xlabel('Área útil (m²)')
plt.ylabel('Preço (R$)')
plt.show()


##  Regressão Linear Múltipla

Construção de modelos com múltiplas variáveis para entender melhor a contribuição de cada fator no preço do imóvel.

In [ ]:

features = ['Size', 'Rooms', 'Toilets', 'Suites', 'Parking', 'Elevator', 'Furnished', 'Swimming Pool', 'New']
X_mult = df[features]
y_mult = df['LogPrice'] 

X_mult_const = sm.add_constant(X_mult)
modelo_multiplo = sm.OLS(y_mult, X_mult_const).fit()
print(modelo_multiplo.summary())


## Diagnóstico e Validação

Foram avaliados: resíduos, normalidade, multicolinearidade (quando duas ou mais variáveis independentes em um modelo de regressão estão altamente correlacionadas) e outras métricas para verificar a validade estatística dos modelos.

In [ ]:

residuos = modelo_multiplo.resid

plt.figure(figsize=(8, 5))
sns.histplot(residuos, kde=True)
plt.title('Distribuição dos Resíduos')
plt.xlabel('Resíduo')
plt.show()

plt.figure(figsize=(8, 5))
plt.scatter(modelo_multiplo.fittedvalues, residuos, alpha=0.3)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Resíduos vs Valores Ajustados')
plt.xlabel('Valores Ajustados')
plt.ylabel('Resíduos')
plt.show()


##  Modelos Avançados

Nesse bloco foram testados: regressão polinomial, interação entre variáveis e/ou aplicação de regularização (Ridge, Lasso) para melhorar o modelo.

In [ ]:

df['Size2'] = df['Size']**2
X_poly = df[['Size', 'Size2']]
X_poly_const = sm.add_constant(X_poly)
modelo_poly = sm.OLS(y, X_poly_const).fit()
print(modelo_poly.summary())
